# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Shi  ->  J. Shi  |  ['J. Shi']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
H. Linz  ->  H. Linz  |  ['H. Linz']


K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Arxiv has 71 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2602.09096
extracting tarball to tmp_2602.09096...

 done.
Retrieving document from  https://arxiv.org/e-print/2602.09513
Retrieving document from  https://arxiv.org/e-print/2602.09544
extracting tarball to tmp_2602.09544...

/tmp/ipykernel_3122/2822249172.py:52: LatexWarning: 2602.09513 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


 done.
Retrieving document from  https://arxiv.org/e-print/2602.09819
extracting tarball to tmp_2602.09819...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2602.10072


extracting tarball to tmp_2602.10072...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2602.10072/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors' from 'tmp_2602.10072/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'inst.tex' from 'tmp_2602.10072/inst.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.

/tmp/ipykernel_3122/2822249172.py:52: LatexWarning: 2602.10072 did not run properly
bad escape \o at position 5
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.09837-b31b1b.svg)](https://arxiv.org/abs/2602.09837) | **Investigating the Nested Structure of the Outflow from the Low Luminosity Protostar IRAS 16253-2429 using JWST and ALMA**  |
|| M. Narang, et al. -- incl., <mark>H. Beuther</mark>, <mark>H. Linz</mark> |
|*Appeared on*| *2026-02-11*|
|*Comments*| *22 pages, 14 figures, in press at AAS*|
|**Abstract**|            Understanding the earliest stage of star and planet formation requires detailed observations to address the connection and interplay between the accretion, outflow, and disk evolution. We present results from observations of the low luminosity ($L_{\mathrm{bol}} \sim 0.2,L_{\odot}$) and low mass ($M_{*} \sim 0.15,M_{\odot}$) Class 0 protostar IRAS~16253$-$2429, conducted as part of the eDisk ALMA Large Program and the JWST Cycle~1 GO program IPA. Observations reveal a wide hourglass-shaped continuum cavity traced in scattered light (at $\leq 5,\mu$m), with a brighter, extended northern side. We detect 15 pure rotational H$2$ transitions ($E{\mathrm{up}} = 1015$--21411~K), revealing a wide-angle molecular outflow. The outflow width traced by H$_2$~0-0~S(11) at the protostellar location measures $\leq 35$~au, slightly larger than the dust and Keplerian disk diameters ($\sim 30$~au) but wider than the 20--23~au jet width in [Fe~II]. The opening angle narrows from 40--35 degrees for the low-$J$ H$_2$ lines (up to S(5)) and the cold gas component traced by ALMA $^{12}$CO to $\sim 28$--19 degrees for the high-$J$ H$_2$ lines (S(7)--S(11)). Position--velocity diagrams of H$2$ reveal higher velocities for higher $E{\mathrm{up}}$, ranging from $\sim 12.5$~km~s$^{-1}$ for H$_2$~0-0~S(1) and S(2) to $\sim 28.5$~km~s$^{-1}$ for H$_2$~0-0~S(5) and S(7) with respect to the mean flow velocity. The nested excitation and velocity structure of the collimated jet and wide-angle wind suggests a magnetohydrodynamic wind as a likely launching mechanism, similar to findings in other protostars and Class~II sources. The lower-velocity millimeter CO emission may be gas from the infalling envelope accelerated outward by the wide-angle wind along the cavity walls.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.09096-b31b1b.svg)](https://arxiv.org/abs/2602.09096) | **A Faint Progenitor System for the Faint Supernova 2024vjm**  |
|| E. A. Zimmerman, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2026-02-11*|
|*Comments*| **|
|**Abstract**|            Type Ia Supernovae (SNe Ia) are well known for their role as standardizable cosmological candles. Their uniformity is credited to their single origin as thermonuclear explosions of White dwarf (WD) stars. Nevertheless, some SNe Ia break this regularity. Prominently, the Iax subclass are less energetic and remarkably diverse, raising questions about their progenitor systems. While no progenitor system of a normal SN Ia has ever been detected, a luminous blue star was identified in pre-explosion images of the site of the bright SN Iax SN 2012Z, suggested to be a helium giant companion star acting as a mass donor to a WD SN progenitor. This is in line with models of weak mass accretion of a WD from a binary companion, producing an explosion that does not fully disrupt the star. However, these models fail to explain the properties of the faintest Type Iax explosions, suggesting either they originate from other WD binary systems, or even from massive progenitor stars. Here, we present the faint SN Iax SN 2024vjm - possibly the faintest supernova observed to date. Using a deep pre-explosion image taken by the recently launched Euclid space mission, we show that its progenitor system must be fainter than the helium giant SN Iax progenitor candidate of SN 2012Z, as well as that of the luminous red companion or remnant of the faint SN 2008ha, and may require a subdwarf helium star as a mass donor. The deep image also provides strong arguments against a massive star origin for this faint supernova. Our observations argue that SN 2024vjm is a WD explosion, but we find that remarkably faint SNe Iax fade more slowly than bright ones, i.e., they evolve in an opposite manner from the famous Phillips relation that makes regular SNe Ia cosmological candles.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.09544-b31b1b.svg)](https://arxiv.org/abs/2602.09544) | **Systematic Study of the Simultaneous Events Detected by GECAM**  |
|| Y.-Z. Ren, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2026-02-11*|
|*Comments*| *15 pages,17 figures*|
|**Abstract**|            GECAM is a constellation of all-sky monitors in hard X-ray and gamma-ray band primarily aimed at high energy transients such as gamma-ray bursts, soft gamma-ray repeaters, solar flares and terrestrial gamma-ray flashes. As GECAM has the highest temporal resolution (0.1~$\mu$s) among instruments of its kind, it can identify the so-called simultaneous events (STE) that deposit signals in multiple detectors nearly at the same time (with a 0.3~$\mu$s window). However, the properties and origin of STE have not yet been explored. In this work, we implemented, for the first time, a comprehensive analysis of the STE detected by GECAM, including their morphology, energy deposition, and the dependence on the geomagnetic coordinates. We find that these STE probably result from direct interactions between high-energy charged cosmic rays and satellite. These results demonstrate that GECAM can detect, identify, and characterize high-energy cosmic rays, making it a Micro Cosmic-Ray Observatory (MICRO) in low Earth orbit.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.09819-b31b1b.svg)](https://arxiv.org/abs/2602.09819) | **Estimating Electron Densities in the Middle Solar Corona using White-light and Radio Observations**  |
|| S. Mondal, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2026-02-11*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            The electron density of the solar corona is a fundamental parameter in many areas of solar physics. Traditionally, routine estimates of coronal density have relied exclusively on white-light observations. However, these density estimates, obtained by inverting the white-light data, require simplifying assumptions, which may affect the robustness of the measurements. Hence, to improve the reliability of coronal density measurements, it is highly desirable to explore other complementary methods. In this study, we estimate the coronal electron densities in the middle corona, between approximately $1.7-3.5R_\odot$, using low-frequency radio observations from the recently commissioned Long Wavelength Array at the Owens Valley Radio Observatory (OVRO-LWA). The results demonstrate consistency with those derived from white-light coronagraph data and predictions from theoretical models. We also derive a density model valid between 1.7--3.5 $r_\odot$ and is given by $\rho (r')=1.27r'^{-2}+29.02r'^{-4}+71.18r'^{-6}$, where $r'=r/R_\odot$, and $r$ is the heliocentric distance. OVRO-LWA is a solar-dedicated radio interferometer that provides science-ready images with low latency, making it well-suited for generating regular and independent estimates of coronal densities to complement existing white-light techniques.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.10072-b31b1b.svg)](https://arxiv.org/abs/2602.10072) | **SDSS-V Local Volume Mapper (LVM): Helix Nebula public data, Data Analysis Pipeline data products**  |
|| S. F. Sánchez, et al. -- incl., <mark>K. Kreckel</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2026-02-11*|
|*Comments*| *20 pages, 12 figures, accepted for publishing in RMxA&A*|
|**Abstract**|            We present a spatially resolved spectroscopic analysis of the Helix Nebula (NGC 7293) using data from the SDSS-V Local Volume Mapper (LVM), by applying the recently developed LVM Data Analysis Pipeline (LVM-DAP). Covering the full optical range (3600-9800 Å) over a contiguous ~ 0.5 degree field, the LVM data provide the first hexagonally sampled, wide-field emission-line maps of all major ionic species in this archetypal planetary nebula. The resulting flux, kinematic, and line-ratio maps reveal the well-known ionization stratification of the nebula, from the compact He++ core to the bright [O III] ring and the extended low-ionization envelope, enabling a detailed comparison with classical aperture spectroscopy. Owing to the sensitivity and uniform spatial sampling of the LVM, numerous faint auroral and diagnostic lines are detected across the nebula, including [O III] 4363, [N II] 5755, and He I lines, allowing precise measurements of weak-line morphology. The derived radial trends confirm the remarkably low dust content and the overall homogeneity of electron temperature and density across the main ring. Ionized-gas kinematics traced by H{\alpha} further support the scenario of a slowly expanding, limb-brightened shell consistent with previous studies. This work demonstrates the diagnostic power of LVM spectroscopy for extended nebulae and highlights its capability to recover both global and spatially resolved physical conditions across complex ionized structures.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \o at position 5</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.09513-b31b1b.svg)](https://arxiv.org/abs/2602.09513) | **A new strategy for achieving multiple continuous cooling stages in an adiabatic demagnetization refrigerator**  |
|| Y. Li, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2026-02-11*|
|*Comments*| *9 pages,5 figures,2 tables*|
|**Abstract**|            This paper presents a three-stage continuous adiabatic demagnetization refrigerator (CADR) simultaneously providing cooling platforms at two different temperatures. Unlike conventional CADR with two continuous stages, this system does not require an extra continuous stage. It is achieved with carefully designed heat switch operations and stage sequences. This new strategy significantly reduces system complexity and mass. The detailed implementation of the two cooling platforms is described, and factors limiting system performance are analyzed. This CADR system has achieved a cooling power of 20 {\mu}W@1 K and 4 {\mu}W@300 mK simultaneously, offering a new design strategy for the development of CADR with multiple continuous stages.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

120  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
